In [1]:
import requests
import pandas as pd

In [2]:
mutation_df = pd.read_csv("MutationsU1.tsv", sep="\t", encoding_errors="ignore")

In [3]:
mutation_df["Start"] -= 1

In [4]:
mutation_df

,Mutation,Chrome,Start,End,Ref,Alt,Germline,Gene
0,chr20:287876-287876_C_T,chr20,287875,287876,C,T,Yes,C20orf96
1,chr20:297347-297347_T_G,chr20,297346,297347,T,G,Yes,NaN
2,chr20:1133834-1133834_A_G,chr20,1133833,1133834,A,G,NaN,PSMF1
3,chr20:1362242-1362242_C_A,chr20,1362241,1362242,C,A,NaN,"ENSG00000274322, (ENSG00000229728)"
4,chr20:1432927-1432927_C_A,chr20,1432926,1432927,C,A,Yes,NaN
...,...,...,...,...,...,...,...,...
216,chr20:63638771-63638771_T_C,chr20,63638770,63638771,T,C,NaN,NaN
217,chr20:63638939-63638939_C_G,chr20,63638938,63638939,C,G,NaN,NaN
218,chr20:63726255-63726255_A_G,chr20,63726254,63726255,A,G,NaN,ZGPAT
219,chr20:63971501-63971501_G_A,chr20,63971500,63971501,G,A,NaN,NaN


In [27]:
# Loop through mutation_df and get the Ref and Alt columns

for index, row in mutation_df.iterrows():
    start = row['Start']
    end = row['End']

    url = f"https://api.genome.ucsc.edu/getData/track?track=dbSnp155;genome=hg38;chrom=chr20;start={start-1};end={end+1}"
    
    response = requests.get(url)

    data = response.json()
    
    try:    
        dbsnp = data["dbSnp155"]
        match_list = []
        for snp in dbsnp:            
            ref = snp["ref"]
            alt = snp["alts"].split(",")
            alt_count = snp["altCount"]
            
            if ref == mutation_df.at[index, "Ref"]:
                match_ref = True
            else:
                match_ref = False
                            
            if any([ax == mutation_df.loc[index, "Alt"] for ax in alt]):
                match_alt = True
            else:
                match_alt = False 
                
            if match_ref and match_alt:
                match_list.append(True)
            else:
                match_list.append(False)
        
        if any(match_list):
            mutation_df.at[index, "dbSNP_germline"] = True
        else:
            mutation_df.at[index, "dbSNP_germline"] = False
            
    except:
        mutation_df.at[index, "dbSNP_germline"] = False

In [28]:
#mutation_df["Ref_UCSC"] = mutation_df["Ref_UCSC"].fillna(mutation_df["Ref"])

In [29]:
# Create a new column called Germline_final where mutation_df["Alt"] substring mutation_df["Alt_UCSC"]
#mutation_df["Germline_final"] = mutation_df.apply(lambda row: row["Alt"] in row["Alt_UCSC"] if not pd.isna(row["Alt_UCSC"]) else False, axis=1)

In [30]:
mutation_df["dbSNP_germline"].value_counts(normalize=True)

True     0.79638
False    0.20362
Name: dbSNP_germline, dtype: float64

In [31]:
mutation_df

,Mutation,Chrome,Start,End,Ref,Alt,Germline,Gene,dbSNP_germline
0,chr20:287876-287876_C_T,chr20,287875,287876,C,T,Yes,C20orf96,True
1,chr20:297347-297347_T_G,chr20,297346,297347,T,G,Yes,NaN,True
2,chr20:1133834-1133834_A_G,chr20,1133833,1133834,A,G,NaN,PSMF1,True
3,chr20:1362242-1362242_C_A,chr20,1362241,1362242,C,A,NaN,"ENSG00000274322, (ENSG00000229728)",True
4,chr20:1432927-1432927_C_A,chr20,1432926,1432927,C,A,Yes,NaN,True
...,...,...,...,...,...,...,...,...,...
216,chr20:63638771-63638771_T_C,chr20,63638770,63638771,T,C,NaN,NaN,True
217,chr20:63638939-63638939_C_G,chr20,63638938,63638939,C,G,NaN,NaN,True
218,chr20:63726255-63726255_A_G,chr20,63726254,63726255,A,G,NaN,ZGPAT,True
219,chr20:63971501-63971501_G_A,chr20,63971500,63971501,G,A,NaN,NaN,True


In [39]:
mutation_df.loc[mutation_df["dbSNP_germline"] == False, :]

,Mutation,Chrome,Start,End,Ref,Alt,Germline,Gene,dbSNP_germline
18,chr20:5163895-5163895_G_A,chr20,5163894,5163895,G,A,NaN,CDS2,False
21,chr20:5532539-5532539_A_G,chr20,5532538,5532539,A,G,NaN,NaN,False
23,chr20:5542035-5542035_T_G,chr20,5542034,5542035,T,G,NaN,NaN,False
24,chr20:5590105-5590105_A_G,chr20,5590104,5590105,A,G,NaN,GPCPD1,False
34,chr20:13773912-13773912_A_G,chr20,13773911,13773912,A,G,NaN,ESF1,False
43,chr20:21755398-21755398_C_T,chr20,21755397,21755398,C,T,NaN,NaN,False
48,chr20:28865386-28865386_G_T,chr20,28865385,28865386,G,T,NaN,NaN,False
49,chr20:28865392-28865392_T_G,chr20,28865391,28865392,T,G,NaN,NaN,False
50,chr20:29101464-29101464_C_G,chr20,29101463,29101464,C,G,NaN,NaN,False
65,chr20:32381742-32381742_T_C,chr20,32381741,32381742,T,C,NaN,ASXL1,False
